# Label Rescaling + Conservative Predictions

**Problem**: CV-LB intercept (0.0525) > Target (0.0347). Even CV=0 would give LB=0.0525.

**Hypothesis**: The intercept represents extrapolation error to unseen solvents. By:
1. **Label rescaling**: Calibrate predictions to match training distribution
2. **Conservative predictions**: Blend toward mean when extrapolating

We might reduce the intercept and change the CV-LB relationship.

**Base model**: exp_030 GP+MLP+LGBM (CV 0.008298, LB 0.0877)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [5]:
# Simple Featurizer (for GP) - 18 features (Spange + Arrhenius kinetics)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Simple feature dimension (for GP): {SimpleFeaturizer().feats_dim}')

Simple feature dimension (for GP): 18


In [6]:
# MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[64, 32], output_dim=3, dropout=0.1):
        super().__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

print('MLP model defined')

MLP model defined


In [7]:
# Label Rescaling + Conservative Predictions Model
class LabelRescalingModel:
    """GP+MLP+LGBM ensemble with label rescaling and conservative predictions."""
    
    def __init__(self, data='single', gp_weight=0.3, mlp_weight=0.4, lgbm_weight=0.3,
                 rescale_alpha=0.5, conservative_alpha=0.3):
        self.data = data
        self.mixed = (data == 'full')
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        self.rescale_alpha = rescale_alpha  # How much to rescale toward training distribution
        self.conservative_alpha = conservative_alpha  # How much to blend toward mean for extrapolation
        
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        
        # Store training statistics for rescaling
        self.train_mean = None
        self.train_std = None
        self.train_features = None  # For extrapolation detection
        
    def train_model(self, X, Y):
        # Store training target statistics for rescaling
        Y_np = Y.values if hasattr(Y, 'values') else Y
        self.train_mean = Y_np.mean(axis=0)
        self.train_std = Y_np.std(axis=0)
        
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        # Store training features for extrapolation detection
        self.train_features = X_full.copy()
        
        # Scale
        X_full_scaled = self.scaler_full.fit_transform(X_full)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple)
        
        # Train GP (on simple features)
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        self.gp_models = []
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2, random_state=42)
            gp.fit(X_simple_scaled, Y_np[:, i])
            self.gp_models.append(gp)
        
        # Train MLP
        X_torch = torch.tensor(X_full_scaled, dtype=torch.double)
        Y_torch = torch.tensor(Y_np, dtype=torch.double)
        
        self.mlp = MLP(input_dim=X_full_scaled.shape[1]).double().to(device)
        optimizer = torch.optim.Adam(self.mlp.parameters(), lr=5e-4, weight_decay=1e-5)
        criterion = nn.HuberLoss()
        
        dataset = TensorDataset(X_torch, Y_torch)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        self.mlp.train()
        for epoch in range(200):
            for xb, yb in loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                pred = self.mlp(xb)
                loss = criterion(pred, yb)
                loss.backward()
                optimizer.step()
        
        # Train LGBM
        self.lgbm_models = []
        for i in range(3):
            model = lgb.LGBMRegressor(
                n_estimators=500, learning_rate=0.03, max_depth=6,
                num_leaves=31, reg_alpha=0.1, reg_lambda=0.1,
                random_state=42, verbose=-1
            )
            model.fit(X_full_scaled, Y_np[:, i])
            self.lgbm_models.append(model)
        
        # Fit nearest neighbors for extrapolation detection
        self.nn_model = NearestNeighbors(n_neighbors=5)
        self.nn_model.fit(X_full_scaled)
        self.train_distances_mean = None  # Will compute on first predict
        
    def predict(self, X):
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        # Scale
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        # GP predictions
        gp_preds = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gp_models])
        
        # MLP predictions
        self.mlp.eval()
        with torch.no_grad():
            X_torch = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
            mlp_preds = self.mlp(X_torch).cpu().numpy()
        
        # LGBM predictions
        lgbm_preds = np.column_stack([m.predict(X_full_scaled) for m in self.lgbm_models])
        
        # Ensemble
        raw_preds = (self.gp_weight * gp_preds + 
                     self.mlp_weight * mlp_preds + 
                     self.lgbm_weight * lgbm_preds)
        
        # Compute extrapolation score (distance to nearest training points)
        distances, _ = self.nn_model.kneighbors(X_full_scaled)
        extrapolation_score = distances.mean(axis=1)
        
        # Compute threshold from training data (if not already computed)
        if self.train_distances_mean is None:
            train_scaled = self.scaler_full.transform(self.train_features)
            train_distances, _ = self.nn_model.kneighbors(train_scaled)
            self.train_distances_mean = train_distances.mean()
            self.train_distances_std = train_distances.std()
        
        # Normalize extrapolation score
        threshold = self.train_distances_mean + 2 * self.train_distances_std
        extrapolation_weight = np.clip(extrapolation_score / threshold, 0, 1).reshape(-1, 1)
        
        # Apply label rescaling: calibrate predictions to match training distribution
        pred_mean = raw_preds.mean(axis=0)
        pred_std = raw_preds.std(axis=0) + 1e-8
        
        # Rescale predictions toward training distribution
        rescaled_preds = raw_preds.copy()
        for i in range(3):
            # Standardize predictions
            standardized = (raw_preds[:, i] - pred_mean[i]) / pred_std[i]
            # Rescale to training distribution
            rescaled = standardized * self.train_std[i] + self.train_mean[i]
            # Blend between raw and rescaled based on rescale_alpha
            rescaled_preds[:, i] = (1 - self.rescale_alpha) * raw_preds[:, i] + self.rescale_alpha * rescaled
        
        # Apply conservative predictions: blend toward mean when extrapolating
        conservative_preds = rescaled_preds.copy()
        for i in range(3):
            # Blend toward training mean based on extrapolation score
            blend_weight = extrapolation_weight * self.conservative_alpha
            conservative_preds[:, i] = (1 - blend_weight.flatten()) * rescaled_preds[:, i] + blend_weight.flatten() * self.train_mean[i]
        
        # Clip to valid range [0, 1]
        final_preds = np.clip(conservative_preds, 0, 1)
        
        return torch.tensor(final_preds, dtype=torch.double)

print('LabelRescalingModel defined')

LabelRescalingModel defined


In [8]:
# Quick test
X_single, Y_single = load_data("single_solvent")
print(f'Single solvent data: X={X_single.shape}, Y={Y_single.shape}')
print(f'Target statistics: mean={Y_single.mean().values}, std={Y_single.std().values}')

Single solvent data: X=(656, 3), Y=(656, 3)
Target statistics: mean=[0.14993233 0.12337957 0.52219232], std=[0.14313596 0.1315279  0.3602294 ]


In [9]:
# Run CV for single solvent data with different alpha values
print('Testing different rescale/conservative alpha combinations...')

X, Y = load_data("single_solvent")

# Test different alpha combinations
alpha_configs = [
    (0.0, 0.0),   # No rescaling, no conservative (baseline)
    (0.1, 0.1),   # Light rescaling
    (0.2, 0.1),   # Moderate rescaling
    (0.1, 0.2),   # Light rescaling, moderate conservative
]

results = []

for rescale_alpha, conservative_alpha in alpha_configs:
    print(f'\nTesting rescale_alpha={rescale_alpha}, conservative_alpha={conservative_alpha}')
    
    split_generator = generate_leave_one_out_splits(X, Y)
    all_predictions = []
    all_actuals = []
    
    for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
        (train_X, train_Y), (test_X, test_Y) = split
        
        model = LabelRescalingModel(data='single', rescale_alpha=rescale_alpha, conservative_alpha=conservative_alpha)
        model.train_model(train_X, train_Y)
        
        predictions = model.predict(test_X)
        predictions_np = predictions.detach().cpu().numpy()
        
        all_predictions.append(predictions_np)
        all_actuals.append(test_Y.values)
    
    all_predictions = np.vstack(all_predictions)
    all_actuals = np.vstack(all_actuals)
    
    mse = np.mean((all_predictions - all_actuals) ** 2)
    results.append((rescale_alpha, conservative_alpha, mse))
    print(f'Single Solvent MSE: {mse:.6f}')

print('\n=== SUMMARY ===')
for r_alpha, c_alpha, mse in results:
    print(f'rescale={r_alpha}, conservative={c_alpha}: MSE={mse:.6f}')

Running CV for single solvent data...


0it [00:00, ?it/s]

1it [00:20, 20.54s/it]

2it [00:41, 20.51s/it]

3it [00:59, 19.79s/it]

...

22it [08:02, 24.23s/it]

23it [08:22, 23.08s/it]

24it [08:43, 22.48s/it]

24it [08:43, 21.82s/it]


Single Solvent MSE: 0.016791


In [10]:
# Run CV for full data
print('Running CV for full data...')

X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = LabelRescalingModel(data='full', rescale_alpha=0.5, conservative_alpha=0.3)
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    all_predictions_full.append(predictions_np)
    all_actuals_full.append(test_Y.values)

all_predictions_full = np.vstack(all_predictions_full)
all_actuals_full = np.vstack(all_actuals_full)

mse_full = np.mean((all_predictions_full - all_actuals_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')

Running CV for full data...


0it [00:00, ?it/s]

1it [01:04, 64.20s/it]

2it [02:09, 64.81s/it]

3it [03:13, 64.40s/it]

...

11it [12:15, 68.92s/it]

12it [13:27, 70.01s/it]

13it [14:28, 67.40s/it]

13it [14:28, 66.85s/it]


Full Data MSE: 0.015121


In [11]:
# Calculate overall CV score
n_single = len(all_actuals)
n_full = len(all_actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBaseline (exp_030 GP+MLP+LGBM): CV 0.008298, LB 0.0877')
print(f'This (Label Rescaling): CV {overall_mse:.6f}')

if overall_mse < 0.008298:
    improvement = (0.008298 - overall_mse) / 0.008298 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than baseline!')
else:
    degradation = (overall_mse - 0.008298) / 0.008298 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than baseline')


=== CV RESULTS ===
Single Solvent MSE: 0.016791 (n=656)
Full Data MSE: 0.015121 (n=1227)
Overall MSE: 0.015703

Baseline (exp_030 GP+MLP+LGBM): CV 0.008298, LB 0.0877
This (Label Rescaling): CV 0.015703

✗ WORSE: 89.23% worse than baseline


In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = LabelRescalingModel(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = LabelRescalingModel(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################